# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Queue Management

**Subtopic** - Simulating a task scheduler with priority using heapq

**Use Case** - Develop a Python module that simulates a task scheduler using a priority queue. The module should include functionalities to add tasks with different priorities, execute tasks based on their priority, and handle cases when multiple tasks have the same priority. Use the heapq library for implementing the priority queue.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

You are building a CPU task scheduler for a backend server that processes various jobs. Each job has a priority level and execution time. The scheduler must ensure high-priority jobs execute before low-priority ones, with same-priority jobs following FIFO order.

Implement a task scheduler that handles three operations: adding jobs, executing jobs for specified time units, and checking queue status.

# Input
A list of operations where each operation is one of:
- `["add", job_id, priority, execution_time]` - Add new job
- `["execute", time_units]` - Execute jobs for given time units  
- `["status"]` - Return current queue status

```
operations = [
    ["add", "job1", 2, 5],
    ["execute", 4],
    ["status"]
]
```

# Output
List of responses for each operation:
- **add**: `"Job {job_id} added"` or `"Error: Job {job_id} already exists"`
- **execute**: List of `[job_id, time_spent]` for jobs completed in this execution
- **status**: List of `[job_id, priority, remaining_time]` ordered by priority

**Examples**

Example 1:
```
Input: [["add", "A", 2, 3], ["add", "B", 1, 2], ["execute", 5]]
Output: ["Job A added", "Job B added", [["B", 2], ["A", 3]]]
```

Example 2:
```
Input: [["add", "X", 1, 4], ["execute", 2], ["add", "Y", 0, 3], ["execute", 5], ["status"]]
Output: ["Job X added", [], "Job Y added", [["Y", 3], ["X", 2]], []]
```

**Function Signature**
```python
def task_scheduler(operations: List[List[Union[str, int]]]) -> List[Union[str, List]]:
    pass
```



# Requirements

**Explicit Requirements**
- Lower priority number = higher priority (0 is highest)
- Same priority jobs follow FIFO order
- Track remaining execution time per job
- Only return jobs completed within each execute operation
- Status returns jobs ordered by execution priority

**Implicit Requirements**  
- Partially executed jobs stay in queue with updated remaining time
- Use arrival counter to maintain FIFO for same priority
- Track job registry to detect duplicates
- Time spent in execute output is for that execution only

**Constraints**
- 1 ≤ operations ≤ 1000
- 0 ≤ priority ≤ 100  
- 1 ≤ execution_time ≤ 100
- Job IDs are unique strings
- Time units are positive integers
- No input validation required

**Edge Cases**
- Execute with 0 time units returns empty list
- Execute on empty queue returns empty list
- Duplicate job ID returns error message
- Empty status returns empty list
- Jobs execute in strict priority order with FIFO tiebreaker

In [ ]:
# code

"""Priority queue task scheduler implementation."""

import heapq
from typing import List, Union


def task_scheduler(
    operations: List[List[Union[str, int]]]
) -> List[Union[str, List]]:
    """
    Implement a priority-based task scheduler using heapq.

    Args:
        operations: List of operations to perform on the scheduler

    Returns:
        List of responses for each operation
    """
    heap = []
    arrival_order = 0
    job_registry = {}
    results = []

    for operation in operations:
        op_type = operation[0]

        if op_type == "add":
            _, job_id, priority, execution_time = operation

            if job_id in job_registry:
                results.append(f"Error: Job {job_id} already exists")
            else:
                heapq.heappush(
                    heap, (priority, arrival_order, job_id, execution_time)
                )
                job_registry[job_id] = True
                arrival_order += 1
                results.append(f"Job {job_id} added")

        elif op_type == "execute":
            time_units = operation[1]
            completed_jobs = []
            remaining_time = time_units

            if time_units == 0:
                results.append([])
                continue

            jobs_in_progress = {}

            while heap and remaining_time > 0:
                priority, order, job_id, job_time = heapq.heappop(heap)

                time_spent = min(remaining_time, job_time)
                remaining_time -= time_spent
                job_time -= time_spent

                if job_id in jobs_in_progress:
                    jobs_in_progress[job_id] += time_spent
                else:
                    jobs_in_progress[job_id] = time_spent

                if job_time > 0:
                    heapq.heappush(heap, (priority, order, job_id, job_time))
                else:
                    completed_jobs.append([job_id, jobs_in_progress[job_id]])
                    del job_registry[job_id]

            results.append(completed_jobs)

        elif op_type == "status":
            queue_status = []
            temp_heap = []

            while heap:
                priority, order, job_id, remaining = heapq.heappop(heap)
                queue_status.append([job_id, priority, remaining])
                temp_heap.append((priority, order, job_id, remaining))

            for item in temp_heap:
                heapq.heappush(heap, item)

            results.append(queue_status)

    return results


In [ ]:
# tests

"""Import libraries and functions."""
import unittest
from main import task_scheduler


class TestTaskScheduler(unittest.TestCase):
    """Unit tests for the task_scheduler function."""

    def test_add_and_execute_basic(self):
        """Test adding jobs and executing them in priority order."""
        ops = [["add", "A", 2, 3], ["add", "B", 1, 2], ["execute", 5]]
        expected = ["Job A added", "Job B added", [["B", 2], ["A", 3]]]
        self.assertEqual(task_scheduler(ops), expected)

    def test_execute_with_zero_time(self):
        """Test execute operation with zero time units returns empty list."""
        ops = [["add", "X", 1, 4], ["execute", 0], ["status"]]
        expected = ["Job X added", [], [["X", 1, 4]]]
        self.assertEqual(task_scheduler(ops), expected)

    def test_duplicate_job_id(self):
        """Test adding a duplicate job ID returns error message."""
        ops = [["add", "A", 1, 2], ["add", "A", 2, 3]]
        expected = ["Job A added", "Error: Job A already exists"]
        self.assertEqual(task_scheduler(ops), expected)

    def test_status_empty_queue(self):
        """Test status operation on empty queue returns empty list."""
        ops = [["status"]]
        expected = [[]]
        self.assertEqual(task_scheduler(ops), expected)

    def test_execute_on_empty_queue(self):
        """Test execute operation on empty queue returns empty list."""
        ops = [["execute", 3]]
        expected = [[]]
        self.assertEqual(task_scheduler(ops), expected)

    def test_priority_and_fifo(self):
        """Test FIFO within same priority; lower number = higher priority."""
        ops = [
            ["add", "A", 1, 2],
            ["add", "B", 1, 3],
            ["add", "C", 0, 1],
            ["execute", 3],
            ["status"]
        ]
        expected = [
            "Job A added",
            "Job B added",
            "Job C added",
            [["C", 1], ["A", 2]],
            [["B", 1, 3]]
        ]
        self.assertEqual(task_scheduler(ops), expected)

    def test_partial_execution_and_remaining(self):
        """Test partial execution and remaining time tracking."""
        ops = [
            ["add", "A", 2, 5],
            ["execute", 3],
            ["status"],
            ["execute", 2],
            ["status"]
        ]
        expected = [
            "Job A added",
            [],
            [["A", 2, 2]],
            [["A", 2]],
            []
        ]
        self.assertEqual(task_scheduler(ops), expected)

    def test_multiple_jobs_varied_priority(self):
        """Test all jobs complete in one execute based on priority."""
        ops = [
            ["add", "A", 3, 2],
            ["add", "B", 2, 2],
            ["add", "C", 1, 2],
            ["add", "D", 0, 2],
            ["execute", 8]
        ]
        expected = [
            "Job A added",
            "Job B added",
            "Job C added",
            "Job D added",
            [["D", 2], ["C", 2], ["B", 2], ["A", 2]]
        ]
        self.assertEqual(task_scheduler(ops), expected)

    def test_execute_time_less_than_job(self):
        """Test when execute time is less than job's full time."""
        ops = [
            ["add", "A", 1, 10],
            ["execute", 4],
            ["status"]
        ]
        expected = [
            "Job A added",
            [],
            [["A", 1, 6]]
        ]
        self.assertEqual(task_scheduler(ops), expected)

    def test_execute_time_exact_job(self):
        """Test when execute time equals job's full execution time."""
        ops = [
            ["add", "A", 1, 4],
            ["execute", 4],
            ["status"]
        ]
        expected = [
            "Job A added",
            [["A", 4]],
            []
        ]
        self.assertEqual(task_scheduler(ops), expected)

    def test_status_ordering(self):
        """Test that status returns correct order of queued jobs."""
        ops = [
            ["add", "A", 2, 2],
            ["add", "B", 1, 2],
            ["add", "C", 1, 1],
            ["status"]
        ]
        expected = [
            "Job A added",
            "Job B added",
            "Job C added",
            [["B", 1, 2], ["C", 1, 1], ["A", 2, 2]]
        ]
        self.assertEqual(task_scheduler(ops), expected)

    def test_large_number_of_operations(self):
        """Test scheduler with 1000 jobs and single execute."""
        ops = []
        for i in range(1, 1001):
            ops.append(["add", f"J{i}", i % 5, 1])
        ops.append(["execute", 1000])
        result = task_scheduler(ops)
        self.assertEqual(result.count("Job J1 added"), 1)
        self.assertEqual(len(result[-1]), 1000)

    def test_add_and_execute_interleaved(self):
        """Test interleaved add and execute operations."""
        ops = [
            ["add", "A", 1, 2],
            ["execute", 1],
            ["add", "B", 0, 1],
            ["execute", 2],
            ["status"]
        ]
        expected = [
            "Job A added",
            [],
            "Job B added",
            [["B", 1], ["A", 1]],
            []
        ]
        self.assertEqual(task_scheduler(ops), expected)

    def test_execute_multiple_jobs_partial(self):
        """Test partial execution with multiple jobs of same priority."""
        ops = [
            ["add", "A", 1, 3],
            ["add", "B", 1, 3],
            ["execute", 4],
            ["status"]
        ]
        expected = [
            "Job A added",
            "Job B added",
            [["A", 3]],
            [["B", 1, 2]]
        ]
        self.assertEqual(task_scheduler(ops), expected)


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/75162/view>

#### Model code:

```python
# code generated by the model

from typing import List, Union, Tuple
import heapq
def task_scheduler(operations: List[List[Union[str, int]]]) -> List[Union[str, List[Union[str, List]]]]:
    pq = []  # priority queue
    job_registry = {}  # to track if a job already exists
    results = []
    arrival_counter = 0
    for op in operations:
        if op[0] == "add":
            # ["add", job_id, priority, execution_time]
            _, job_id, priority, execution_time = op
            
            if job_id in job_registry:
                results.append(f"Error: Job {job_id} already exists")
            else:
                heapq.heappush(pq, (priority, arrival_counter, job_id, execution_time))
                job_registry[job_id] = (priority, arrival_counter, execution_time)
                arrival_counter += 1
                results.append(f"Job {job_id} added")
        elif op[0] == "execute":
            # ["execute", time_units]
            _, time_units = op
            
            completed_jobs = []
            time_remaining = time_units
            # To handle the jobs we cannot fully execute, we will need a temp list
            temp_pq = []
            
            while pq and time_remaining > 0:
                priority, arrival_order, job_id, remaining_time = heapq.heappop(pq)
                
                if remaining_time <= time_remaining:
                    # Complete this job
                    completed_jobs.append([job_id, remaining_time])
                    time_remaining -= remaining_time
                    del job_registry[job_id]
                else:
                    # Partially complete this job
                    completed_jobs.append([job_id, time_remaining])
                    remaining_time -= time_remaining
                    temp_pq.append((priority, arrival_order, job_id, remaining_time))
                    # <Issue>: job_registry is not updated with the new remaining_time after partial execution
                    time_remaining = 0
            
            # Restore the partially executed jobs back into the queue
            for item in temp_pq:
                heapq.heappush(pq, item)
            results.append(completed_jobs)
        elif op[0] == "status":
            # ["status"]
            current_status = [[job_id, priority, remaining_time] for priority, arrival_order, job_id, remaining_time in pq]
            # <Issue>: sort() sorts by job_id (first element) instead of priority, violating "Status returns jobs ordered by execution priority"
            # <Issue>: Even if sorted by priority, this loses FIFO order for same-priority jobs, violating "Same priority jobs follow FIFO order"
            current_status.sort()  # already sorted by priority and arrival_order
            results.append(current_status)
    
    return results
```